In [1]:
import time
import numpy as np
import pandas as pd
import os

from hydesign.assembly.hpp_assembly import hpp_model
from hydesign.examples import examples_filepath

In [2]:
sites = pd.read_csv('overview.csv', index_col=0)
example = 0

ex_site = sites.iloc[int(example),:]

print('Selected example site:')
print('---------------------------------------------------')
print(ex_site.T)

longitude = ex_site['longitude']
latitude = ex_site['latitude']
altitude = ex_site['altitude']
input_ts_fn = os.path.join(os.getcwd(), ex_site['input_ts_fn'])
sim_pars_fn = os.path.join(os.getcwd(), ex_site['sim_pars_fn'])

hpp = hpp_model(
        latitude,
        longitude,
        altitude,
        num_batteries = 10,
        work_dir = './',
        sim_pars_fn = sim_pars_fn,
        input_ts_fn = input_ts_fn,
        save_battry_rf_filename='battery_rf.csv',
)

inputs = dict(
clearance =	60,
sp =	287.6391934,
p_rated =	10,
Nwt	=	25,
wind_MW_per_km2 =	4.782274215,
solar_MW =	223.2191223,
surface_tilt =	50,
surface_azimuth = 210,
DC_AC_ratio	=1.295598428,
b_P =75.27402422,
b_E_h=	4.451612903,
cost_of_battery_P_fluct_in_peak_price_ratio	=0.580645161,)

Selected example site:
---------------------------------------------------
case                                              Europe
name                                    France_good_wind
longitude                                      -0.864258
latitude                                       48.744116
altitude                                             302
input_ts_fn      hyd_input/input_ts_France_good_wind.csv
sim_pars_fn                       hyd_input/hpp_pars.yml
price_fn                     hyd_input/2030-EL_PRICE.csv
price_col                                           FR_R
H2_demand_col                    hyd_input/H2_demand.csv
Name: 0, dtype: object

Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302

input_ts_fn length is not a complete number of years (hyDesign handles years as 365 days).
The file has been modified and stored in ./input_ts_modified.csv


In [3]:
start = time.time()
outs = hpp.evaluate(**inputs)
hpp.print_design([v for k,v in inputs.items()], outs)
end = time.time()
print('exec. time [min]:', (end - start)/60 )
print(hpp.prob['NPV_over_CAPEX'])


Design:
---------------
clearance [m]: 60.000
sp [W/m2]: 287.639
p_rated [MW]: 10.000
Nwt: 25.000
wind_MW_per_km2 [MW/km2]: 4.782
solar_MW [MW]: 223.219
surface_tilt [deg]: 50.000
surface_azimuth [deg]: 210.000
DC_AC_ratio: 1.296
b_P [MW]: 75.274
b_E_h [h]: 4.452
cost_of_battery_P_fluct_in_peak_price_ratio: 0.581


NPV_over_CAPEX: 1.060
NPV [MEuro]: 455.417
IRR: 0.128
LCOE [Euro/MWh]: 32.548
Revenues [MEuro]: 46.767
CAPEX [MEuro]: 429.684
OPEX [MEuro]: 7.260
Wind CAPEX [MEuro]: 275.586
Wind OPEX [MEuro]: 5.958
PV CAPEX [MEuro]: 65.197
PV OPEX [MEuro]: 1.301
Batt CAPEX [MEuro]: 22.236
Batt OPEX [MEuro]: 0.000
Shared CAPEX [MEuro]: 66.665
Shared Opex [MEuro]: 0.000
penalty lifetime [MEuro]: 0.000
AEP [GWh]: 1163.214
GUF: 0.443
grid [MW]: 300.000
wind [MW]: 250.000
solar [MW]: 223.219
Battery Energy [MWh]: 335.091
Battery Power [MW]: 75.274
Total curtailment [GWh]: 86.031
Awpp [km2]: 52.276
Apvp [km2]: 2.737
Plant area [km2]: 52.276
Rotor diam [m]: 210.393
Hub height [m]: 165.196
Number 

In [4]:
from hopp.simulation import HoppInterface

/home/cbay/tool-benchmarking/Europe/log/hybrid_systems_2024-05-12T16.31.36.216823.log


In [9]:
hi = HoppInterface(os.path.join(os.getcwd(), "hopp_input/08-wind-solar-france.yaml"))

FLORIS is the system model...


In [10]:
hi.simulate(25)

Simulating wind farm output in FLORIS...


In [11]:
hybrid_plant = hi.system

annual_energies = hybrid_plant.annual_energies
npvs = hybrid_plant.net_present_values
cf = hybrid_plant.capacity_factors

wind_installed_cost = hybrid_plant.wind.total_installed_cost
solar_installed_cost = hybrid_plant.pv.total_installed_cost
battery_installed_cost = hybrid_plant.battery.total_installed_cost
hybrid_installed_cost = hybrid_plant.grid.total_installed_cost

print("Wind Installed Cost [M$]: {}".format(wind_installed_cost/1e6))
print("Solar Installed Cost [M$]: {}".format(solar_installed_cost/1e6))
print("Battery Installed Cost [M$]: {}".format(battery_installed_cost/1e6))
print("Hybrid Installed Cost [M$]: {}\n".format(hybrid_installed_cost/1e6))

print("Wind NPV [M$]: {}".format(hybrid_plant.net_present_values.wind/1e6))
print("Solar NPV [M$]: {}".format(hybrid_plant.net_present_values.pv/1e6))
print("Hybrid NPV [M$]: {}\n".format(hybrid_plant.net_present_values.hybrid/1e6))

print("Annual Energies [kW]",annual_energies)
print("Capacity Factors",cf)
print("NPV [$]", npvs)

print("LCOE [cents/kWh]", hybrid_plant.lcoe_real,"\n")

print("Hybrid LCOE [$/MWh]", hybrid_plant.lcoe_real['hybrid']*10)

Wind Installed Cost [M$]: 241.227
Solar Installed Cost [M$]: 54.989034800000006
Battery Installed Cost [M$]: 17.5693665
Hybrid Installed Cost [M$]: 313.7854013

Wind NPV [M$]: 43.13380019290936
Solar NPV [M$]: 2.545612164523393
Hybrid NPV [M$]: 39.73664301620609

Annual Energies [kW] {"pv": 277789162.6089397, "wind": 663116990.2109435, "battery": -2859748.7759711565, "hybrid": 936837216.2571286}
Capacity Factors {"pv": 18.486286595799378, "wind": 30.279314621504266, "battery": 0, "hybrid": 21.565886860864747}
NPV [$] {"pv": 2545612.164523393, "wind": 43133800.19290936, "battery": -2140217.4290492404, "hybrid": 39736643.016206086}
LCOE [cents/kWh] {"pv": 2.4773060421576454, "wind": 2.9739200415014664, "battery": 5.048881089434874, "hybrid": 3.1005842519559565} 

Hybrid LCOE [$/MWh] 31.005842519559565


In [7]:
import json

with open("singleowner.json", 'w') as f:
    dat = hybrid_plant.grid._financial_model.export()
    d = dict()
    for k, v in dat.items():
        d.update(v)
    json.dump(d, f)